In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import sys
import random
import math
import re
import time
import numpy as np
import cv2
import matplotlib
import matplotlib.pyplot as plt
import yaml

In [ ]:
%matplotlib inline 

In [ ]:
from model.mrcnn.config import Config
from model.mrcnn import utils
import model.mrcnn.model as modellib
from model.mrcnn import visualize
from model.mrcnn.model import log
from model.mrcnn import data_generator as datagen
from model.mrcnn import image_meta as meta
from model.mrcnn import dataset

In [ ]:
from data.pascalvoc.pascal_frcnn import PascalVOCDataset

In [ ]:
from config.config import cfg
from config import config_utils

In [ ]:
from pprint import pprint

In [ ]:
EXP_FOLDER = '/home/caleml/partial_experiments/exp_20200214_1749_frcnn_pascal_extended_frcnn_pascal_extended'

In [ ]:
def get_ax(rows=1, cols=1, size=8):
    """
    Return a Matplotlib Axes array to be used in
    all visualizations in the notebook. Provide a
    central point to control graph sizes.
    
    Change the default size attribute to control the size
    of rendered images
    """
    _, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
    return ax

# Config

In [ ]:
config_path = os.path.join(EXP_FOLDER, 'config.yaml')
with open(config_path, 'r') as f_in:
    loaded_conf = yaml.load(f_in)

config_utils.update_config(loaded_conf)
pprint(cfg)

# Dataset

In [ ]:
# Validation dataset
dataset_val = PascalVOCDataset()
dataset_val.load_pascal(cfg.DATASET.PATH, 8, 'val', extended=True, cfg=cfg)
dataset_val.prepare()

# model

In [ ]:
cfg.BATCH_SIZE = 1

In [ ]:
model = modellib.MaskRCNN(mode='inference', 
                          cfg=cfg,
                          exp_folder=EXP_FOLDER)

In [ ]:
checkpoint_path = model.find_last()

In [ ]:
print("Loading weights from ", checkpoint_path)

In [ ]:
model.load_weights(checkpoint_path, by_name=True)

# examples

## gt image

In [ ]:
image_id = random.choice(dataset_val.image_ids)
print(image_id)
original_image, image_meta, gt_class_id, gt_bbox = datagen.load_image_gt(dataset_val, cfg, image_id)

log("original_image", original_image)
log("image_meta", image_meta)
log("gt_class_id", gt_class_id)
log("gt_bbox", gt_bbox)

visualize.display_instances(original_image, gt_bbox, gt_class_id, dataset_val.class_names, figsize=(8, 8))

## detected image

In [ ]:
results = model.detect([original_image], verbose=1)

print(results)

r = results[0]
visualize.display_instances(original_image, r['rois'], r['class_ids'], dataset_val.class_names, r['scores'], ax=get_ax())

# eval

In [ ]:
# Compute VOC-Style mAP @ IoU=0.5
# Running on 10 images. Increase for better accuracy.
# image_ids = np.random.choice(dataset_val.image_ids, 1000)
image_ids = dataset_val.image_ids

APs = []
count = 0

for image_id in image_ids:
    count += 1
    if count % 1000 == 0:
        print("done %s val images, temp AP %s" % (count, np.mean(APs)))
    
    # Load image and ground truth data
    image, image_meta, gt_class_id, gt_bbox = datagen.load_image_gt(dataset_val, cfg, image_id)
    
    # Run object detection
    results = model.detect([image], verbose=0)
    r = results[0]
    
    # Compute AP
    AP, precisions, recalls, overlaps = utils.compute_ap(gt_bbox, gt_class_id, r["rois"], r["class_ids"], r["scores"])
    APs.append(AP)
    
print("mAP: ", np.mean(APs))

In [ ]:
print(len(APs))